In [1]:
import collections

from intcode import Machine

In [2]:
with open("day17.input") as file:
    prog = file.readline().strip()

In [3]:
def find_neighbours(pos):
    neighbours = []
    for candidate in [
        (pos[0], pos[1] + 1),
        (pos[0], pos[1] - 1),
        (pos[0] + 1, pos[1]),
        (pos[0] - 1, pos[1])]:
        if grid.get(candidate) == '#':
            neighbours.append(candidate)
    return neighbours

In [4]:
def get_picture(robot):
    row, col = 0, 0
    grid = dict()
    for char in robot._outputs:
        if chr(char) in ['^', 'v', '>', '<']:
            robot_pos = (row, col)
        if char == 10:
            row += 1
            col = 0
        else:
            grid[(row, col)] = chr(char)
            col += 1
    return grid, robot_pos

In [5]:
def draw_grid(grid):
    max_x = max((k[0] for k in grid))
    min_x = min((k[0] for k in grid))
    max_y = max((k[1] for k in grid))
    min_y = min((k[1] for k in grid))

    for x in range(min_x, max_x + 1):
        row = ''.join(grid.get((x, y)) for y in range(min_y, max_y + 1))
        print(row)

# Part 1

In [6]:
m = Machine(prog)
m.run()

[INFO] intcode(180): Machine 0 is halted


In [7]:
grid, start = get_picture(m)

In [8]:
queue = collections.deque([start])
visited = set()
intersect = set()

while queue:
    pos = queue.pop()
    visited.add(pos)
    
    neighbours = find_neighbours(pos)
    if len(neighbours) == 4:
        intersect.add(pos)
        
    for n in neighbours:
        if not n in visited:
            queue.appendleft(n)

In [9]:
sum([x*y for x, y in intersect])

7816

In [10]:
draw_grid(grid)

............................#......................
............................#......................
..........#######...........#......................
..........#.....#...........#......................
..........#.....#...........#.......###########....
..........#.....#...........#.......#.........#....
..........#.....#...........#############.....#....
..........#.....#...................#...#.....#....
..........#.....#.....#############.#...#.....#....
..........#.....#.....#...........#.#...#.....#....
..........#.....#.....#...........#.#############..
..........#.....#.....#...........#.....#.....#.#..
..........#.....#############.....#.....#.....#.#..
..........#...........#.....#.....#.....#.....#.#..
^##########...........#.###########.....#.....#.#..
......................#.#...#...........#.....#.#..
......................#.#...#...........#######.#..
......................#.#...#...................#..
......................#######...............#######
............

# Part 2

In [11]:
#  0
# 3 1
#  2
heading_to_int = {
    '^': 0,
    'v': 2,
    '>': 1,
    '<': 3,
}

int_to_heading = {v: k for k, v in heading_to_int.items()}

In [12]:
def find_turn(pos, heading):
    candidate = int_to_heading[(heading_to_int[heading] + 1) % 4]
    if can_move_forward(pos, candidate):
        return 'R', candidate
    candidate = int_to_heading[(heading_to_int[heading] - 1) % 4]
    if can_move_forward(pos, candidate):
        return 'L', candidate
    raise StopIteration("Cannot turn either L or R")

In [13]:
def in_front_of(pos, heading):
    if heading == '^':
        return (pos[0] - 1, pos[1])
    if heading == 'v':
        return (pos[0] + 1, pos[1])
    if heading == '>':
        return (pos[0], pos[1] + 1)
    if heading == '<':
        return (pos[0], pos[1] - 1)

In [14]:
def can_move_forward(pos, heading):
    if grid.get(in_front_of(pos, heading)) == '#':
        return True
    return False

In [15]:
def find_track(pos, heading):
    track = []
    while grid[pos]:
        counter = 0
        while can_move_forward(pos, heading):
            counter += 1
            pos = in_front_of(pos, heading)
        if counter > 0:
            track.append(str(counter))   

        try:
            turn, heading = find_turn(pos, heading)
        except StopIteration:
            return track
        track.append(turn)

In [16]:
def get_all_outputs(machine):
    output = []
    while machine._outputs:
        output.append(machine.get_output())
    return output

In [17]:
def send_input(sequence, machine):
    for c in sequence:
        m.add_input(ord(c))
    m.add_input(10)

In [18]:
track = find_track(start, grid[start])
path = ','.join(map(str, track))
path

'R,10,L,12,R,6,R,10,L,12,R,6,R,6,R,10,R,12,R,6,R,10,L,12,L,12,R,6,R,10,R,12,R,6,R,10,L,12,L,12,R,6,R,10,R,12,R,6,R,10,L,12,L,12,R,6,R,10,R,12,R,6,R,10,L,12,R,6'

In [19]:
A = "R,10,L,12,R,6"
B = "R,6,R,10,R,12,R,6"
C = "R,10,L,12,L,12"
main = path.replace(A, 'A').replace(B, "B").replace(C, "C")
main

'A,A,B,C,B,C,B,C,B,A'

In [20]:
m = Machine(prog)
m.set_mem_value(address=0, value=2)

m.run()
print(''.join([chr(i) for i in get_all_outputs(m)]))

[INFO] intcode(140): Machine 0: Waiting for new input


............................#......................
............................#......................
..........#######...........#......................
..........#.....#...........#......................
..........#.....#...........#.......###########....
..........#.....#...........#.......#.........#....
..........#.....#...........#############.....#....
..........#.....#...................#...#.....#....
..........#.....#.....#############.#...#.....#....
..........#.....#.....#...........#.#...#.....#....
..........#.....#.....#...........#.#############..
..........#.....#.....#...........#.....#.....#.#..
..........#.....#############.....#.....#.....#.#..
..........#...........#.....#.....#.....#.....#.#..
^##########...........#.###########.....#.....#.#..
......................#.#...#...........#.....#.#..
......................#.#...#...........#######.#..
......................#.#...#...................#..
......................#######...............#######
............

In [21]:
# Send main routine
send_input(main, m)

# Send functions A, B and C
send_input(A, m)
send_input(B, m)
send_input(C, m)

# Send No
send_input("n", m)

m.run()
print(''.join([chr(i) for i in m._outputs]))

[INFO] intcode(180): Machine 0 is halted


Function A:
Function B:
Function C:
Continuous video feed?

............................^......................
............................#......................
..........#######...........#......................
..........#.....#...........#......................
..........#.....#...........#.......###########....
..........#.....#...........#.......#.........#....
..........#.....#...........#############.....#....
..........#.....#...................#...#.....#....
..........#.....#.....#############.#...#.....#....
..........#.....#.....#...........#.#...#.....#....
..........#.....#.....#...........#.#############..
..........#.....#.....#...........#.....#.....#.#..
..........#.....#############.....#.....#.....#.#..
..........#...........#.....#.....#.....#.....#.#..
###########...........#.###########.....#.....#.#..
......................#.#...#...........#.....#.#..
......................#.#...#...........#######.#..
......................#.#...#...................#..
....

In [22]:
m._outputs.pop()

952010